In [16]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# --- Function to plot each context ---
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def plot_each_context(profiler_data, outdir=None, file_format="png", do_show=False):
    """
    Plot each context in a separate figure with bar + pie charts.
    Optionally save each figure in the specified format (png or svg).

    Args:
        profiler_data (dict): Nested profiling data
        outdir (str): Directory to save figures. If None, figures are only shown.
        file_format (str): Target file format, "png" or "svg". Default is "png".
    """
    if outdir is not None:
        os.makedirs(outdir, exist_ok=True)

    if file_format.lower() not in ["png", "svg"]:
        raise ValueError("file_format must be 'png' or 'svg'")
    results = {} # {context: fig}

    for ctx, ctx_data in profiler_data.items():
        summary = ctx_data["step_dict"]["summary"]
        avg_times = summary["avg_time"]
        percent_times = summary["percent_time"]

        # Create figure with 1 row, 2 columns
        fig = make_subplots(
            rows=1,
            cols=2,
            subplot_titles=[f"{ctx} - Avg Time", f"{ctx} - % Time"],
            specs=[[{"type": "bar"}, {"type": "pie"}]],
        )

        # Bar chart
        fig.add_trace(
            go.Bar(
                x=list(avg_times.keys()),
                y=list(avg_times.values()),
                text=[f"{v*1000:.2f} ms" for v in avg_times.values()],
                textposition="outside",
                name="Avg Time",
            ),
            row=1,
            col=1,
        )

        # Pie chart
        fig.add_trace(
            go.Pie(
                labels=list(percent_times.keys()),
                values=list(percent_times.values()),
                hole=0.4,
                name="% Time",
            ),
            row=1,
            col=2,
        )

        fig.update_layout(
            title_text=f"Profiler Summary: {ctx}",
            width=1000,
            height=400,
            showlegend=True,
        )

        # Show figure
        if do_show:
            fig.show()

        # Save figure if outdir is provided
        if outdir is not None:
            file_path = os.path.join(outdir, f"{ctx}_summary.{file_format.lower()}")
            fig.write_image(file_path)
            print(f"Saved figure: {file_path}")

        results[ctx] = fig
    return results


# --- Test data ---
test_data = {
    "pipeline": {
        "duration": 0.960867,
        "step_dict": {
            "summary": {
                "avg_time": {
                    "avg_read_frame": 0.00558,
                    "avg_big_model_infer": 0.03710,
                },
                "percent_time": {
                    "per_read_frame": 13.08,
                    "per_big_model_infer": 86.92,
                },
                "total_avg_time": 0.04268,
            },
            "detail": {},
        },
    },
    "big_model_infer": {
        "duration": 0.93146,
        "step_dict": {
            "summary": {
                "avg_time": {
                    "avg_preprocess": 0.01042,
                    "avg_infer": 0.02666,
                },
                "percent_time": {
                    "per_preprocess": 28.10,
                    "per_infer": 71.89,
                },
                "total_avg_time": 0.03709,
            },
            "detail": {},
        },
    },
    "post_process": {
        "duration": 0.20012,
        "step_dict": {
            "summary": {
                "avg_time": {
                    "avg_decode": 0.05012,
                    "avg_resize": 0.02000,
                },
                "percent_time": {
                    "per_decode": 71.0,
                    "per_resize": 29.0,
                },
                "total_avg_time": 0.07012,
            },
            "detail": {},
        },
    },
}

# --- Run test ---
plot_each_context(test_data, do_show=True, outdir="profiler_plots", file_format="png")

Saved figure: profiler_plots/pipeline_summary.png


Saved figure: profiler_plots/big_model_infer_summary.png


Saved figure: profiler_plots/post_process_summary.png


{'pipeline': Figure({
     'data': [{'name': 'Avg Time',
               'text': [5.58 ms, 37.10 ms],
               'textposition': 'outside',
               'type': 'bar',
               'x': [avg_read_frame, avg_big_model_infer],
               'xaxis': 'x',
               'y': [0.00558, 0.0371],
               'yaxis': 'y'},
              {'domain': {'x': [0.55, 1.0], 'y': [0.0, 1.0]},
               'hole': 0.4,
               'labels': [per_read_frame, per_big_model_infer],
               'name': '% Time',
               'type': 'pie',
               'values': [13.08, 86.92]}],
     'layout': {'annotations': [{'font': {'size': 16},
                                 'showarrow': False,
                                 'text': 'pipeline - Avg Time',
                                 'x': 0.225,
                                 'xanchor': 'center',
                                 'xref': 'paper',
                                 'y': 1.0,
                                 'yanchor': 'b